### Reinitialize the file

In [1]:
# warning !! remove all except this script
## remove -i argument to run
!rm `(ls | grep -v '.ipynb')`

rm: cannot remove 'demo_transform_structure': Is a directory


In [2]:
pdbid='2jpz'
drug='POH'
jobname = f'{pdbid}_{drug}' if not drug is None else pdbid

In [3]:
import os
ff_folder  = os.path.realpath("../../../smsl/opt/force_field")
mdp_folder = os.path.realpath("../../../smsl/opt/mdp")

### G-quadruplex Prepare

In [4]:
!cp ../../c.gromacs.run_md/1.create_system/*{drug}* ./
!cp ../../c.gromacs.run_md/1.create_system/*.itp ./
!cp ../../c.gromacs.run_md/1.create_system/*.top ./
!cp ../../c.gromacs.run_md/1.create_system/{pdbid}.gro.pdb ./
!cp ../5.Output/{jobname}.gro.pdb ./

In [5]:
# 畫完box後座標會平移
# 因此最後要將座標再align回原本座標
# 因為不建立水箱 因此 -d 數值隨意(但不要太小以防止因為pbc而導致結構被切一半) 
!gmx editconf -f {jobname}.gro.pdb -o {jobname}_dode.pdb -c yes -d 1.5 -bt dodecahedron

                     :-) GROMACS - gmx editconf, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii_poh/b.amber.drug_ff/6.Em_min
Command line:
  gmx editconf -f 2jpz_POH.gro.pdb -o 2jpz_POH_dode.pdb -c yes -d 1.5 -bt dodecahedron

Note that major changes are planned in future for editconf, to improve usability and utility.
Read 807 atoms
No velocities found
    system size :  2.520  2.938  2.445 (nm)
    diameter    :  3.084               (nm)
    center      :  0.048 -0.074 -0.042 (nm)
    box vectors :  0.000  0.000  0.000 (nm)
    box angles  :   0.00   0.00   0.00 (degrees)
    box volume  :   0.00               (nm^3)
    shift       :  4.515  4.637  2.193 (nm)
new center      :  4.563  4.563  2.151 (nm)
new box vectors :  6.084  6.084  6.084 (nm)
new box angles  :  60.00  60.00  90.00 (degrees)
new box volume  : 159.25               (nm^3)

GROMACS reminds you: "We Lo

In [3]:
cmd = f'''\
r {drug} | r DG
name 5 {drug}_DG
q
'''
!echo -e "{cmd}" | gmx make_ndx -f {jobname}.gro.pdb -o {jobname}.ndx

             :-) GROMACS - gmx make_ndx, 2021.4-Ubuntu-2021.4-2 (-:

                            GROMACS is written by:
     Andrey Alekseenko              Emile Apol              Rossen Apostolov     
         Paul Bauer           Herman J.C. Berendsen           Par Bjelkmar       
       Christian Blau           Viacheslav Bolnykh             Kevin Boyd        
     Aldert van Buuren           Rudi van Drunen             Anton Feenstra      
    Gilles Gouaillardet             Alan Gray               Gerrit Groenhof      
       Anca Hamuraru            Vincent Hindriksen          M. Eric Irrgang      
      Aleksei Iupinov           Christoph Junghans             Joe Jordan        
    Dimitrios Karkoulis            Peter Kasson                Jiri Kraus        
      Carsten Kutzner              Per Larsson              Justin A. Lemkul     
       Viveca Lindahl            Magnus Lundborg             Erik Marklund       
        Pascal Merz             Pieter Meulenhoff           

### 因為藥物跟loop空間上會重疊，因此將藥物往下位移一點

In [4]:
cmd=f'''\
pymol >>
load {jobname}_dode.pdb
extract {drug}, resn {drug}
show_as sticks
color green  , resn DA and name *C*
color red    , resn DT and name *C*
color magenta, resn DG and name *C*
color cyan  , resn POH and name *C*
color wheat , resn EWV and name *C*
sele {drug}
[click by mouse] (sele) > action > drag_coordinate > 
    shift+middle mouse button to [translate drug]
    shift+right  mouse button to [rotate drug] > Done
set retain_order, 1
save {jobname}_dode.1.pdb, enable
>> EOF
'''
print(cmd)

pymol >>
load 2jpz_POH_dode.pdb
extract POH, resn POH
show_as sticks
color green  , resn DA and name *C*
color red    , resn DT and name *C*
color magenta, resn DG and name *C*
color cyan  , resn POH and name *C*
color wheat , resn EWV and name *C*
sele POH
[click by mouse] (sele) > action > drag_coordinate > 
    shift+middle mouse button to [translate drug]
    shift+right  mouse button to [rotate drug] > Done
set retain_order, 1
save 2jpz_POH_dode.1.pdb, enable
>> EOF



## First Cycle of EM1

In [8]:
## 固定藥物和核心區域(DG)
freeze_cmd = f'''\
freezegrps      = {drug}_DG
freezedim       = Y Y Y
'''

In [9]:
!echo "{freeze_cmd}"        > min1.mdp
!cat {mdp_folder}/min1.mdp >> min1.mdp

In [10]:
!gmx grompp -f min1.mdp -c {jobname}_dode.1.pdb -r {jobname}_dode.1.pdb -p topol.top -o min1.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min1 -c min1.pdb -v

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii_poh/b.amber.drug_ff/6.Em_min
Command line:
  gmx grompp -f min1.mdp -c 2jpz_POH_dode.1.pdb -r 2jpz_POH_dode.1.pdb -p topol.top -o min1.tpr -n 2jpz_POH.ndx -maxwarn 1

Setting the LD random seed to -43019009

Generated 5050 of the 5050 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 5050 of the 5050 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'Ion2'

Excluding 3 bonded neighbours molecule type 'POH'

NOTE 1 [file topol.top, line 62]:
  System has non-zero total charge: -15.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.




WARNING 1

### First cycle of EM2

In [11]:
!echo "{freeze_cmd}"        > min2.mdp
!cat {mdp_folder}/min1.mdp >> min2.mdp

In [12]:
!gmx grompp -f min2.mdp -c min1.pdb -r {jobname}_dode.1.pdb -p topol.top -o min2.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min2 -c min2.pdb -v

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii_poh/b.amber.drug_ff/6.Em_min
Command line:
  gmx grompp -f min2.mdp -c min1.pdb -r 2jpz_POH_dode.1.pdb -p topol.top -o min2.tpr -n 2jpz_POH.ndx -maxwarn 1

Setting the LD random seed to -687870235

Generated 5050 of the 5050 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 5050 of the 5050 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'Ion2'

Excluding 3 bonded neighbours molecule type 'POH'

NOTE 1 [file topol.top, line 62]:
  System has non-zero total charge: -15.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.




WARNING 1 [file top

### Second cycle
### 現在上層loop往上移一點了，因此我們可以再將藥物往上移一點

In [6]:
cmd=f'''\
pymol >>
load min2.pdb
extract {drug}, resn {drug}
show_as sticks
color green  , resn DA and name *C*
color red    , resn DT and name *C*
color magenta, resn DG and name *C*
color cyan  , resn POH and name *C*
color wheat , resn EWV and name *C*
sele {drug}
[click by mouse] (sele) > action > drag_coordinate > 
    shift+middle mouse button to [translate drug]
    shift+right  mouse button to [rotate drug] > Done
set retain_order, 1
save {jobname}_dode.2.pdb, enable
>> EOF
'''
print(cmd)

pymol >>
load min2.pdb
extract POH, resn POH
show_as sticks
color green  , resn DA and name *C*
color red    , resn DT and name *C*
color magenta, resn DG and name *C*
color cyan  , resn POH and name *C*
color wheat , resn EWV and name *C*
sele POH
[click by mouse] (sele) > action > drag_coordinate > 
    shift+middle mouse button to [translate drug]
    shift+right  mouse button to [rotate drug] > Done
set retain_order, 1
save 2jpz_POH_dode.2.pdb, enable
>> EOF



### Second cycle of EM1

In [14]:
!gmx grompp -f min1.mdp -c {jobname}_dode.2.pdb -r {jobname}_dode.2.pdb -p topol.top -o min1.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min1 -c min1.pdb -v

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii_poh/b.amber.drug_ff/6.Em_min
Command line:
  gmx grompp -f min1.mdp -c 2jpz_POH_dode.2.pdb -r 2jpz_POH_dode.2.pdb -p topol.top -o min1.tpr -n 2jpz_POH.ndx -maxwarn 1

Setting the LD random seed to -1350569284

Generated 5050 of the 5050 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 5050 of the 5050 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'Ion2'

Excluding 3 bonded neighbours molecule type 'POH'

NOTE 1 [file topol.top, line 62]:
  System has non-zero total charge: -15.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.




WARNING

### Second cycle of EM2

In [15]:
!gmx grompp -f min2.mdp -c min1.pdb -r {jobname}_dode.2.pdb -p topol.top -o min2.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min2 -c min2.pdb -v

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii_poh/b.amber.drug_ff/6.Em_min
Command line:
  gmx grompp -f min2.mdp -c min1.pdb -r 2jpz_POH_dode.2.pdb -p topol.top -o min2.tpr -n 2jpz_POH.ndx -maxwarn 1

Setting the LD random seed to -168039490

Generated 5050 of the 5050 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 5050 of the 5050 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'Ion2'

Excluding 3 bonded neighbours molecule type 'POH'

NOTE 1 [file topol.top, line 62]:
  System has non-zero total charge: -15.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.




WARNING 1 [file top

### Third cycle
### 現在將藥物換成原本的pose再做最後一次能量最小化

In [5]:
cmd=f'''\
pymol >>
load min2.pdb
extract {drug}_min2, min2 and resn {drug}
load {jobname}_dode.pdb
extract {drug}, {jobname} and resn {drug}
show_as sticks
color green  , resn DA and name *C*
color red    , resn DT and name *C*
color magenta, resn DG and name *C*
color cyan  , resn POH and name *C*
color wheat , resn EWV and name *C*
set retain_order, 1
save {jobname}_dode.3.pdb, min2 or {drug}
>> EOF
'''
print(cmd)

pymol >>
load min2.pdb
extract POH_min2, min2 and resn POH
load 2jpz_POH_dode.pdb
extract POH, 2jpz_POH and resn POH
show_as sticks
color green  , resn DA and name *C*
color red    , resn DT and name *C*
color magenta, resn DG and name *C*
color cyan  , resn POH and name *C*
color wheat , resn EWV and name *C*
set retain_order, 1
save 2jpz_POH_dode.3.pdb, min2 or POH
>> EOF



### Third cycle of EM1

In [17]:
!gmx grompp -f min1.mdp -c {jobname}_dode.3.pdb -r {jobname}_dode.3.pdb -p topol.top -o min1.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min1 -c min1.pdb -v

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii_poh/b.amber.drug_ff/6.Em_min
Command line:
  gmx grompp -f min1.mdp -c 2jpz_POH_dode.3.pdb -r 2jpz_POH_dode.3.pdb -p topol.top -o min1.tpr -n 2jpz_POH.ndx -maxwarn 1

Setting the LD random seed to -2297171

Generated 5050 of the 5050 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 5050 of the 5050 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'Ion2'

Excluding 3 bonded neighbours molecule type 'POH'

NOTE 1 [file topol.top, line 62]:
  System has non-zero total charge: -15.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.




WARNING 1 

### Second cycle of EM2

In [18]:
!gmx grompp -f min2.mdp -c min1.pdb -r {jobname}_dode.2.pdb -p topol.top -o min2.tpr -n {jobname}.ndx -maxwarn 1
!gmx mdrun -deffnm min2 -c min2.pdb -v

                      :-) GROMACS - gmx grompp, 2023.2 (-:

Executable:   /home/user/opt/gromacs-2023/bin/gmx
Data prefix:  /home/user/opt/gromacs-2023
Working dir:  /home/user/workfolder/g4dna_protocol/hybrid-ii_poh/b.amber.drug_ff/6.Em_min
Command line:
  gmx grompp -f min2.mdp -c min1.pdb -r 2jpz_POH_dode.2.pdb -p topol.top -o min2.tpr -n 2jpz_POH.ndx -maxwarn 1

Setting the LD random seed to -51234850

Generated 5050 of the 5050 non-bonded parameter combinations
Generating 1-4 interactions: fudge = 0.5

Generated 5050 of the 5050 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'DNA'

Excluding 3 bonded neighbours molecule type 'Ion2'

Excluding 3 bonded neighbours molecule type 'POH'

NOTE 1 [file topol.top, line 62]:
  System has non-zero total charge: -15.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.




WARNING 1 [file topo

In [19]:
cmd=f'''\
pymol >>
load min2.pdb
load {jobname}.gro.pdb
align min2, {jobname}.gro
set retain_order, 1
save {jobname}.final.gro.pdb, min2
>> EOF
'''
print(cmd)

pymol >>
load min2.pdb
load 2jpz_POH.gro.pdb
align min2, 2jpz_POH.gro
set retain_order, 1
save 2jpz_POH.final.gro.pdb, min2
>> EOF



最終 {jobname}.final.gro.pdb 就是分子模擬的初始結構